# Code source du chapitre 2
*mis à jour le 01.07.2022*

In [1]:
import os

from chapitre_1_dump import import_data_batiments

# Récupération et prétraitement des données du chapitre 1
PATH = os.path.join("_DATA","volumesbatisparis_ensapvs.geojson")
volumes = import_data_batiments(PATH,h_etage=3)

volumes

,geometry,surface,hauteur,hauteur_paf
0,"POLYGON ((2.38462 48.82753, 2.38463 48.82753, ...",281.214948,12.0,None
1,"POLYGON ((2.38475 48.82722, 2.38476 48.82721, ...",9.842935,27.0,"[[18, 27]]"
2,"POLYGON ((2.38493 48.82732, 2.38495 48.82730, ...",52.982843,30.0,"[[0, 6], [12, 30]]"
3,"POLYGON ((2.38354 48.82665, 2.38323 48.82652, ...",229.771164,27.0,"[[9, 27]]"
4,"POLYGON ((2.38304 48.82670, 2.38315 48.82659, ...",310.513295,30.0,"[[12, 30]]"
...,...,...,...,...
330,"POLYGON ((2.38469 48.82720, 2.38469 48.82720, ...",1.632114,15.0,"[[9, 15]]"
331,"POLYGON ((2.38451 48.82752, 2.38452 48.82751, ...",3.082522,12.0,"[[0, 3], [9, 12]]"
332,"POLYGON ((2.38453 48.82753, 2.38454 48.82752, ...",5.338095,18.0,"[[0, 6], [15, 18]]"
333,"POLYGON ((2.38463 48.82775, 2.38464 48.82774, ...",4.296431,18.0,"[[15, 18]]"


## 1. Génération de graphiques statistiques

In [2]:
import plotly.graph_objects as go

# Génération d'un graphique de répartition des hauteurs des volumes
def exporter_graphique_hauteurs(data,output_name):

    # Extraction du nombre d'entités pour chaque valeur du champ "hauteur"
    n_par_hauteur = data["hauteur"].value_counts().to_dict()
    # Paramétrage du graphique
    fig = go.Figure([
        go.Bar(x=list(n_par_hauteur.keys()), 
        y=list(n_par_hauteur.values()),
        opacity=0.8,
        marker_color='rgb(200,0,0)')
        ])
    fig.update_xaxes(
        categoryorder='category ascending',
        tickvals=sorted(list(n_par_hauteur.keys()))
        )
    fig.update_layout(
        title="Répartition des hauteurs",
        xaxis_title="hauteur (m)",
        yaxis_title="nombre de volumes",
        width=600,
        height=600
        )
    # Export sous forme de page web statique
    fig.write_html(os.path.join("_OUTPUT",output_name))
    return fig

graph_hauteurs = exporter_graphique_hauteurs(volumes,"volumesbatisparis_ensapvs_hauteurs.html")
graph_hauteurs

## 2. Génération de cartes interactives


In [3]:
def exporter_carte_interactive(data,output_name):

    # Regroupement des volumes par hauteur
    volumes_par_hauteur = data.groupby("hauteur")

    # Liste des couches (par hauteur)
    traces = []
    for h,volumes in volumes_par_hauteur:
        couleur = "rgb(" + ",".join([str(h/max(data["hauteur"])*255)]*3) +")"
        Y, X = [],[]
        for geom in volumes["geometry"]:
            coords = geom.exterior.coords.xy
            X += list(coords[0][:-1])+[None] # longitude
            Y += list(coords[1][:-1])+[None] # latitude
        
        traces.append(go.Scattermapbox(
            name=str(h) +"m",
            mode="lines",
            line={"width" : 0.5, "color" : couleur},
            lon=X,
            lat=Y,
            opacity=1.0,
            hoverinfo="name",
            fill="toself")
            )

    fig = go.Figure(traces)
    fig.update_layout(
        title="Hauteurs des volumes bâtis",legend_title="Hauteur", autosize=True,
        mapbox = {
            'style' : "carto-positron",
            'center' : {'lon': 2.3848515, 'lat': 48.8272092},
            "zoom" : 16.6
            }
        )
        
    # Export sous forme d'une page web
    fig.write_html(os.path.join("_OUTPUT",output_name))
    return fig

cartes_des_hauteurs = exporter_carte_interactive(volumes,"volumesbatisparis_ensapvs_carte.html")
cartes_des_hauteurs

## 3. Génération d'un dessin au format .DXF (CAO)

In [4]:
import ezdxf

def exporter_plan_dxf(data,output_name):

    # Coordonnées latitude/longitude (WGS84)
    data.crs = 4326
    # Coordonnées X/Y (Lambert93)
    data = data.to_crs(2154)
    # initialisation d'un nouveau dessin
    doc = ezdxf.new(dxfversion='R2010')
    msp = doc.modelspace()
    # Récupération et tri par ordre croissant des différentes hauteurs
    hauteurs = sorted(data["hauteur"].unique())
    # Itération à travers les différentes hauteurs
    for h in hauteurs:
        # Création d'un nouveau calque nommé
        calque = doc.layers.new(str(h) + "m")
        # Affectation d'une couleur suivant la hauteur
        calque.rgb = (255*(h/hauteurs[-1]),0,0)

    for row,volume in data.iterrows():
        # Extraction des coordonnées des points du polygone
        coords = list(zip(*volume["geometry"].exterior.coords.xy))
        # Affectation au sein du calque correspondant à sa hauteur
        calque = str(volume["hauteur"]) + "m"
        msp.add_polyline2d(coords, dxfattribs={'layer': calque})
        
    doc.saveas(os.path.join("_OUTPUT",output_name))

exporter_plan_dxf(volumes,"volumesbatisparis_ensapvs_CAD.dxf")

## 4. Génération d'un modèle 3D (format .3dm)

[rhino3dm](https://pypi.org/project/rhino3dm/) est ici préféré à l'ancienne bibliothèque [rhinoinside](https://pypi.org/project/rhinoinside/).

In [20]:
import rhino3dm as r3d

def generer_extrusion_dans_calque(coords, h0, h1, id_calque):
    pts = [r3d.Point3d(c[0],c[1],h0) for c in coords]
    curve = r3d.Curve.CreateControlPointCurve(pts,1)
    obj = r3d.Extrusion.Create(curve,h1,True)
    attr = r3d.ObjectAttributes()
    attr.LayerIndex = id_calque
    return obj,attr

def generer_objet_rhino(volume, id_calque, doc):
    # Extraction des coordonnées
    coords = list(zip(*volume["geometry"].exterior.coords.xy))
    objs = []
    if not volume["hauteur_paf"]:
        obj,attr = generer_extrusion_dans_calque(coords,0.0,-1*volume["hauteur"],id_calque)
        doc.Objects.AddExtrusion(obj,attr)
    else:
        for hspan in volume["hauteur_paf"]:
            obj,attr = generer_extrusion_dans_calque(coords,hspan[0],-1*(hspan[1]-hspan[0]),id_calque)
            doc.Objects.AddExtrusion(obj,attr)
    return objs

def exporter_modele_3dm(data,output_name):
    # Conversion des coordonnées en Lambert 93
    data.crs = 4326
    data = data.to_crs(2154)

    # Initialisation d'un nouveau modèle
    doc = r3d.File3dm()
    # Définition d'un calque 'bâtiment'
    calque_bati = r3d.Layer()
    id_calque_bati = doc.Layers.AddLayer("batiments",(255,0,0,0))

    # Modélisation à partir des coordonnées des volumes
    data.apply(generer_objet_rhino, axis=1, id_calque=id_calque_bati, doc=doc)
    doc.Write(os.path.join("_OUTPUT",output_name))

exporter_modele_3dm(volumes,"volumesbatisparis_ensapvs_3D.3dm")

*ENSAPVS*

![ensapvs](_imgs/3D_ensapvs.jpg)